In [1]:
import os
import sys
import json
import torch
import datetime
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Ensure paths are correct for imports
sys.path.append('..')
from experiments.cross_val import run_cv_experiment 
from loaders.eeg_loader import load_eeg_dataset
import config


MODELS = ["CustomEEGNet", "EEGNet", "DeepConvNet", "ShallowConvNet"]
LRS = [1e-3, 5e-4]
BATCH_SIZES = [16, 32]


tuning_grid = {
    "Original": [None],
    "ChannelsDropout": [0.3, 0.5, 0.7],
    "FTSurrogate": [1.57, 3.14, 6.28], 
    "SmoothTimeMask": [150, 300, 500],
    "TimeReverse": [True]
}

# --- 2. DATA LOAD ---
X, y, metadata, n_classes = load_eeg_dataset(mode="single", subject_id=1)
if X is None:
    print(f"ERROR: Data not found. Check relative paths. CWD: {os.getcwd()}")
    sys.exit()

# Zero Leakage: Split indices defined BEFORE any augmentation
groups_for_cv = metadata['trial_ids']
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results_file = "overnight_search_results.json"
if os.path.exists(results_file):
    with open(results_file, 'r') as f:
        all_results = json.load(f)
    print(f"Resuming from {len(all_results)} existing results.")
else:
    all_results = []

# --- 3. THE AUTOMATION LOOP ---
for model_name in MODELS:
    for lr in LRS:
        for bs in BATCH_SIZES:
            # Consistent hyperparams for fair comparison
            hparams = {
                "batch_size": bs,
                "lr": lr,
                "epochs": 20,       # Increased slightly for better convergence
                "weight_decay": 1e-3, 
                "data_multiplier": 4 # Expansion factor for stochastic dataset
            }

            for method, values in tuning_grid.items():
                for val in values:
                    run_id = f"{model_name}_{method}_{val}_LR{lr}_BS{bs}"
                    
                    # Skip if already calculated (Crash Recovery)
                    if any(res['run_id'] == run_id for res in all_results):
                        continue

                    print(f"\n>>> TESTING: {run_id} | {datetime.datetime.now().strftime('%H:%M:%S')}")

                    # Mapping the grid to the specific dict structure get_augmentation expects
                    current_aug = {}
                    if method != "Original":
                        # Convert CamelCase to snake_case for the internal dictionary keys
                        aug_key = method.lower().replace("channelsdropout", "channels_dropout") \
                                               .replace("ftsurrogate", "freq_surrogate") \
                                               .replace("smoothtimemask", "smooth_time_mask") \
                                               .replace("timereverse", "time_reverse")
                        
                        # Map the specific param name required for each function
                        param_name = "p_drop" if "Dropout" in method else \
                                     "phase_noise_max" if "FT" in method else \
                                     "mask_len_samples" if "Mask" in method else "active"
                        
                        current_aug = {aug_key: {param_name: val}}

                    try:
                        history, mean_acc = run_cv_experiment(
                            X, y, groups_for_cv, n_classes, cv, 
                            exp_name=method, 
                            aug_params=current_aug, 
                            hyperparams=hparams,
                            model_name=model_name,
                            verbose=False
                        )

                        all_results.append({
                            "run_id": run_id, 
                            "accuracy": float(mean_acc), 
                            "model": model_name, 
                            "method": method,
                            "val": val, 
                            "lr": lr, 
                            "bs": bs
                        })
                        
                        # Save after every successful run to prevent data loss
                        with open(results_file, 'w') as f:
                            json.dump(all_results, f, indent=4)
                            
                    except Exception as e:
                        print(f"!!! FAILED {run_id}: {str(e)}")
                        continue

print("\n>>> ALL OVERNIGHT EXPERIMENTS FINISHED. RESULTS SAVED.")


>>> TESTING: CustomEEGNet_Original_None_LR0.001_BS16 | 04:19:49


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


 -> Fold 1 Finished. Best Val Acc: 23.44%
 -> Fold 2 Finished. Best Val Acc: 25.00%
 -> Fold 3 Finished. Best Val Acc: 21.88%
 -> Fold 4 Finished. Best Val Acc: 26.56%
 -> Fold 5 Finished. Best Val Acc: 20.31%

>>> TESTING: CustomEEGNet_ChannelsDropout_0.3_LR0.001_BS16 | 04:31:12


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


 -> Fold 1 Finished. Best Val Acc: 23.44%
 -> Fold 2 Finished. Best Val Acc: 23.44%
 -> Fold 3 Finished. Best Val Acc: 25.00%
 -> Fold 4 Finished. Best Val Acc: 26.56%
 -> Fold 5 Finished. Best Val Acc: 15.62%

>>> TESTING: CustomEEGNet_ChannelsDropout_0.5_LR0.001_BS16 | 04:43:25


...

 -> Fold 1 Finished. Best Val Acc: 25.00%
 -> Fold 2 Finished. Best Val Acc: 26.56%
 -> Fold 3 Finished. Best Val Acc: 31.25%
 -> Fold 4 Finished. Best Val Acc: 21.88%
 -> Fold 5 Finished. Best Val Acc: 12.50%

>>> TESTING: EEGNet_ChannelsDropout_0.5_LR0.001_BS32 | 15:57:47


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


 -> Fold 1 Finished. Best Val Acc: 21.88%
 -> Fold 2 Finished. Best Val Acc: 25.00%
 -> Fold 3 Finished. Best Val Acc: 25.00%
 -> Fold 4 Finished. Best Val Acc: 28.12%
 -> Fold 5 Finished. Best Val Acc: 17.19%

>>> TESTING: EEGNet_ChannelsDropout_0.7_LR0.001_BS32 | 16:09:55


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


KeyboardInterrupt: 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import config
from experiments.cross_val import run_cv_experiment
from utils.io_utils import plot_history, plot_tuning_results
import matplotlib.pyplot as plt
from loaders.eeg_loader import load_eeg_dataset
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut

import optuna
import datetime

experiments = ["Non_Augmented", "ChannelsDropout", "FTSurrogate", "TimeReverse", "SmoothTimeMask"]
results = {}

In [2]:


# ==============================================================================
#   SETUP: LOAD DATA ONCE (Crucial for Speed)
# ==============================================================================
# We load data outside the loop so we don't reload it 100 times.
DATA_MODE = "single"  
SUBJECT_ID = 1        

print(f"Loading Data ({DATA_MODE})...")
X, y, metadata, n_classes = load_eeg_dataset(mode=DATA_MODE, subject_id=SUBJECT_ID)

if DATA_MODE == "loso":
    groups = metadata["subject_ids"]
    splitter = LeaveOneGroupOut()
else:
    groups = metadata["trial_ids"]
    splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # 3 folds is faster for tuning

# ==============================================================================
#   OPTUNA OBJECTIVE FUNCTION
# ==============================================================================
def objective(trial):
    """
    Optuna calls this function repeatedly. 
    It suggests parameters, runs the model, and returns accuracy.
    """
    
    # --- SUGGEST MODEL ---
    model_name = trial.suggest_categorical("model", ["SpectroTemporalTransformer", "EEGNet", "CustomEEGNet"])
    
    # --- SUGGEST HYPERPARAMETERS ---
    # Log-uniform search for LR finds the right order of magnitude quickly
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True) 
    bs = trial.suggest_categorical("batch_size", [16, 32])
    
    # Construct the hyperparam dict
    hparams = {
        "lr": lr,
        "batch_size": bs,
        "epochs": 15,           # Keep epochs lower for tuning speed (e.g., 15 vs 30)
        "weight_decay": 1e-4,
        "data_multiplier": 4    # Keep fixed or suggest_int(2, 6)
    }
    
    # --- SUGGEST AUGMENTATION STRATEGY ---
    # Pick a method
    aug_method = trial.suggest_categorical("aug_method", [
        "Original", 
        "ChannelsDropout", 
        "SmoothTimeMask", 
        "FreqSurrogate",
        "TimeReverse"
    ])
    
    aug_params = None
    
    # 2. Tune the specific parameters for that method
    if aug_method == "ChannelsDropout":
        # Let Optuna find the exact best dropout rate (e.g., 0.342 vs 0.3)
        p_drop = trial.suggest_float("p_drop", 0.1, 0.6)
        aug_params = {"channels_dropout": {"p_drop": p_drop}}
        
    elif aug_method == "SmoothTimeMask":
        # Tune the mask length
        mask_len = trial.suggest_int("mask_len_samples", 50, 500)
        aug_params = {"smooth_time_mask": {"mask_len_samples": mask_len}}
        
    elif aug_method == "FreqSurrogate":
        # Tune the phase noise intensity
        phase_std = trial.suggest_float("phase_noise_std", 0.5, 6.28)
        aug_params = {"freq_surrogate": {"phase_noise_std": phase_std}}
        
    elif aug_method == "TimeReverse":
        aug_params = {"time_reverse": {"active": True}}

    # --- D. RUN EXPERIMENT ---
    try:
        # We call your existing trainer directly
        # Note: run_cv_experiment returns (history, mean_acc)
        _, mean_acc = run_cv_experiment(
            X, y, groups, n_classes, splitter,
            exp_name=aug_method,
            aug_params=aug_params,
            hyperparams=hparams,
            model_name=model_name,
            verbose=False # Keep it quiet
        )
        return mean_acc

    except Exception as e:
        print(f"Trial failed: {e}")
        return 0.0 # Return 0 so Optuna avoids these params

# ==============================================================================
#   RUN OPTIMIZATION
# ==============================================================================
# Create a study object and optimize the objective function.
study = optuna.create_study(direction="maximize", study_name="EEG_Optimization")

print("\n>>> STARTING OPTUNA SEARCH <<<")
# n_trials=50 means it will run 50 distinct experiments
study.optimize(objective, n_trials=50, timeout=None) 

# ==============================================================================
#   REPORT RESULTS
# ==============================================================================
print("\n" + "="*40)
print("BEST TRIAL RESULTS")
print("="*40)
best = study.best_trial

print(f"Best Accuracy: {best.value:.4%}")
print("Best Parameters:")
for key, value in best.params.items():
    print(f"  {key}: {value}")

# Optional: Save best params to a JSON file for your records
import json
with open("optuna_best_params.json", "w") as f:
    json.dump(best.params, f, indent=4)

Loading Data (single)...


[I 2026-02-06 17:08:17,140] A new study created in memory with name: EEG_Optimization
c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(



>>> STARTING OPTUNA SEARCH <<<


[I 2026-02-06 17:36:17,466] Trial 0 finished with value: 0.22497501910303885 and parameters: {'model': 'EEGNet', 'lr': 0.0012114731520183023, 'batch_size': 16, 'aug_method': 'TimeReverse'}. Best is trial 0 with value: 0.22497501910303885.
c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
[I 2026-02-06 17:36:18,152] Trial 1 finished with value: 0.0 and parameters: {'model': 'SpectroTemporalTransformer', 'lr': 0.0015860194835951333, 'batch_size': 16, 'aug_method': 'SmoothTimeMask', 'mask_len_samples': 400}. Best is trial 0 with value: 0.22497501910303885.


Trial failed: local variable 'F' referenced before assignment


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
[I 2026-02-06 18:22:52,267] Trial 2 finished with value: 0.25944865690942215 and parameters: {'model': 'CustomEEGNet', 'lr': 0.002760672775332356, 'batch_size': 16, 'aug_method': 'SmoothTimeMask', 'mask_len_samples': 165}. Best is trial 2 with value: 0.25944865690942215.
c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
[W 2026-02-06 18:55:33,954] Trial 3 failed with parameters: {'model': 'CustomEEGNet', 'lr': 0.0014046765430343762, 'batch_size': 16, 'aug_method': 'TimeReverse'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\optuna\study\_optimize.py", line 206, in _run_trial
    value_or_values = fun

KeyboardInterrupt: 

In [ ]:

tuning_grid = {
    "ChannelsDropout": {
        "p_drop": [0.1, 0.3, 0.5, 0.7, 0.9] 
    },
    "FTSurrogate": {
        "phase_noise_max": [0.5, 1.5, 3.14, 4.71, 6.28]  
    },
    "SmoothTimeMask": {
        "mask_len_samples": [102, 204, 307, 410, 512] 
    }
}


In [ ]:
X, y, groups, n_classes = load_eeg_dataset(mode="single", subject_id=1)
groups_for_cv = groups['trial_ids'] # This should be length 320

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
tuning_output = []


print(">>> Running Baseline (Non_Augmented)...")

_, base_acc = run_cv_experiment(X, y, groups_for_cv, n_classes, cv, "Original", {}, config.HYPERPARAMS)
tuning_output.append({'method': 'Baseline', 'params': 0, 'accuracy': base_acc})


for method, grid in tuning_grid.items():
    param_name = list(grid.keys())[0]
    values = grid[param_name]
    
    for val in values:
        print(f"\n>>> Testing {method} | {param_name}: {val}")
        
        
        current_params = {method.lower(): {param_name: val}}
        
        
        history, mean_acc = run_cv_experiment(
            X, y, groups_for_cv, n_classes, cv, 
            exp_name=method, 
            aug_params=current_params, 
            hyperparams=config.HYPERPARAMS,
            verbose=False 
        )
        
        tuning_output.append({
            'method': method,
            'params': val,
            'accuracy': mean_acc,
            'history': history
        })



>>> Running Baseline (Non_Augmented)...


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


 -> Fold 1 Finished. Best Val Acc: 14.06% (Epoch 16)
 -> Fold 2 Finished. Best Val Acc: 20.31% (Epoch 41)
 -> Fold 3 Finished. Best Val Acc: 15.62% (Epoch 18)
 -> Fold 4 Finished. Best Val Acc: 17.19% (Epoch 29)
 -> Fold 5 Finished. Best Val Acc: 18.75% (Epoch 37)

>>> Testing ChannelsDropout | p_drop: 0.1


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


 -> Fold 1 Finished. Best Val Acc: 15.62% (Epoch 50)
 -> Fold 2 Finished. Best Val Acc: 17.19% (Epoch 3)
 -> Fold 3 Finished. Best Val Acc: 23.44% (Epoch 45)
 -> Fold 4 Finished. Best Val Acc: 25.00% (Epoch 49)
 -> Fold 5 Finished. Best Val Acc: 20.31% (Epoch 20)

>>> Testing ChannelsDropout | p_drop: 0.3


...

 -> Fold 1 Finished. Best Val Acc: 20.31% (Epoch 27)
 -> Fold 2 Finished. Best Val Acc: 20.31% (Epoch 18)
 -> Fold 3 Finished. Best Val Acc: 20.31% (Epoch 16)
 -> Fold 4 Finished. Best Val Acc: 15.62% (Epoch 12)
 -> Fold 5 Finished. Best Val Acc: 25.00% (Epoch 30)

>>> Testing SmoothTimeMask | mask_len_samples: 307


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


 -> Fold 1 Finished. Best Val Acc: 17.19% (Epoch 6)
 -> Fold 2 Finished. Best Val Acc: 18.75% (Epoch 47)
 -> Fold 3 Finished. Best Val Acc: 17.19% (Epoch 36)
 -> Fold 4 Finished. Best Val Acc: 10.94% (Epoch 15)
 -> Fold 5 Finished. Best Val Acc: 21.88% (Epoch 25)

>>> Testing SmoothTimeMask | mask_len_samples: 410


c:\Users\Lova\.conda\envs\IS_env\lib\site-packages\sklearn\model_selection\_split.py:883: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


KeyboardInterrupt: 

In [ ]:

plot_tuning_results(tuning_output)